In [ ]:
import pandas as pd
import numpy as np

# READING THE DATASET AND MAKING CHANGING THE INDEX

In [ ]:
df = pd.read_csv('../input/covid19-confirmed-cases-kerala/Covid 19 Confirmed Cases-Kerala.csv',index_col='Date',parse_dates=True)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.index.freq='D'

In [ ]:
df.index

In [ ]:
df.plot(figsize=(13,8))

# CHECKING MOVING AVERAGES

In [ ]:
df_rolling=df.rolling(window=31).mean()

In [ ]:
df_rolling

In [ ]:

df['Confirmed'].plot(figsize=(13,8),legend=True)
df_rolling['Confirmed'].plot(legend=True,color='red');

In [ ]:
df['Confirmed']=df['Confirmed'].astype(float)

# REMOVNIG INITIAL VALUES CLOSE TO 0

In [ ]:
df=df.loc['2020-06-01':]

In [ ]:
df

In [ ]:
df_rolling=df.rolling(window=7).mean()

In [ ]:
df['Confirmed'].plot(figsize=(13,8),legend=True)
df_rolling['Confirmed'].plot(legend=True,color='red');

VISUALLY NOT MUCH STATIONARITY IS VISIBLE

# RUNNING ETS(ERROR TREND SEASONALITY) DECOMPOSITION

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose  

In [ ]:
result = seasonal_decompose(df['Confirmed'],model='mul')

In [ ]:
result.plot();

THERE IS VERY LOW SEASONALITY
BUT SOME INCREASING GENERAL TREND

# AUGMENTED DICKEY FULLER TEST FOR CHECKING STATIONARITY

In [ ]:
pip install statsmodels

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def adf_test(series,title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC') # .dropna() handles differenced data
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out[f'critical value ({key})']=val
        
    print(out.to_string())          # .to_string() removes the line "dtype: float64"
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")

In [ ]:
adf_test(df['Confirmed'])

# DATASET IS NON STATIONARY p value >0.05 

In [ ]:
len(df)

In [ ]:
train = df.iloc[:418]
test = df.iloc[418:]

# Holt Winters Method

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

NOT PASSING SEASONAL PERIODS AS IT CANNOT BE COMMENTED CLEARLY

In [ ]:
model = ExponentialSmoothing(train['Confirmed'],trend='mul',seasonal='mul')

In [ ]:
fitted_model = model.fit()

In [ ]:
trained_values = fitted_model.fittedvalues.rename('trained_values')

In [ ]:
trained_values

In [ ]:
train['Confirmed'].plot(figsize=(13,8),legend=True)
trained_values.plot(legend=True)


# CHECKING PREDICTIONS ON TEST DATA

In [ ]:
predictions = fitted_model.forecast(5).rename('predictions')

In [ ]:
test['Confirmed'].plot(figsize=(13,8),legend=True)
predictions.plot(figsize=(13,8),legend=True)

In [ ]:
train[-5:].plot(figsize=(13,8))
test['Confirmed'].plot(figsize=(13,8))
predictions.plot(figsize=(13,8))

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test['Confirmed'], predictions)

In [ ]:
df['Confirmed'].describe()

In [ ]:
error

LOOKS ACCEPTABLE 

# ARIMA

In [ ]:
pip install  pmdarima

In [ ]:
from pmdarima import auto_arima

# IGNORING SMALL SEASONALITY FOR NOW

# MAKING ACF,PACF PLOTS TO GET ROUGH IDEA FOR p,q,d

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
title = 'Autocorrelation: Confirmed cases'
lags = 40
plot_acf(df['Confirmed'],title=title,lags=lags);

In [ ]:
title = 'Partial Autocorrelation: Confirmed cases'
lags = 40
plot_pacf(df['Confirmed'],title=title,lags=lags);

SHARP DROP IN PACF PLOT AFTER LAG OF 2 => p should be around 1
CONTINOUSLY DECREASING AUTO CRRELATION VALUES WITH TIME=> Q should be around 0

# GRID SEARCH FOR BEST POSSIBLE p,q,d

In [ ]:
stepwise_fit = auto_arima(df['Confirmed'],start_p=0,start_q=0,seasonal=False,trace=True)

In [ ]:
stepwise_fit.summary()

# ARIMA MODEL WITH p=2,d=1,q=5

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(df['Confirmed'],order=(2,1,5))
fitted_model = model.fit()
fcast = fitted_model.predict(len(train),len(train)+len(test)-1,typ='levels').rename('ARIMA(2,1,5) Predicted')

In [ ]:
fcast

In [ ]:
fcast.plot(figsize=(13,8),legend=True)
test['Confirmed'].plot(legend=True)

# LOOKS MUCH BETTER FIT

In [ ]:
from statsmodels.tools.eval_measures import rmse

error = rmse(test['Confirmed'], fcast)

In [ ]:
error

about 1/3 times less error than that of Holtwinters

# FORECASTING INTO UNKNOWN FUTURE

In [ ]:
df.tail()

SINCE THE ANALYSIS IS DONE ON 31-07-2021
WE CAN ACTUALLY CHECK 5 DAYS LATER HOW MUCH OUR FORECAST MATCHES

In [ ]:
forecast = fitted_model.predict(len(df),len(df)+5,typ='levels').rename('ARIMA(2,1,5) Future')

In [ ]:
forecast

In [ ]:
forecast.plot(figsize=(13,8))

In [ ]:
forecast.plot(figsize=(13,8))
df['Confirmed'].plot(figsize=(13,8))

WILL NEED TO CHECK HOW MUCH THESE PREDICTIONS MATCH ON 3rd AUGUST 2021